In [1]:
!pip install numpy pandas matplotlib ta streamlit yfinance prophet plotly

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2ee9da41193418ec1d3ce10df758a02a01e27f98cc202d4aaa851e130d89628e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
!pip install pyngrok

In [3]:
%%writefile technical_indicators.py
# Create a new file called technical_indicators.py to separate the technical analysis functions

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# Function to calculate trends
def trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

# Function to calculate RSI buy and sell signals
def rsi_signals(df):
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    df['Buy_Signal_RSI'] = df['RSI'] < 40
    df['Sell_Signal_RSI'] = df['RSI'] > 70
    return df

# Function to calculate Bollinger Bands buy and sell signals
def bollinger_signals(df):
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['Buy_Signal_BB'] = df['Close'] < df['BB_Lower']
    df['Sell_Signal_BB'] = df['Close'] > df['BB_Upper']
    return df

# Function to calculate MACD buy and sell signals
def macd_signals(df):
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['Buy_Signal_MACD'] = df['MACD'] > df['MACD_Signal']
    df['Sell_Signal_MACD'] = df['MACD'] < df['MACD_Signal']
    return df

# Function to apply all technical indicators
def apply_all_indicators(df):
    """Calculate all technical indicators and signals"""
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))
    df = rsi_signals(df)
    df = bollinger_signals(df)
    df = macd_signals(df)

    # Add moving averages
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    return df

# Function to plot RSI with Buy/Sell Signals
def plot_rsi(df, figsize=(12, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    ax.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    ax.scatter(df.index[df['Buy_Signal_RSI']], df['RSI'][df['Buy_Signal_RSI']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_RSI']], df['RSI'][df['Sell_Signal_RSI']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Relative Strength Index (RSI)')
    ax.set_ylabel('RSI Value')
    ax.legend()
    ax.grid(True, alpha=0.3)
    return fig

# Function to plot Bollinger Bands with Buy/Sell Signals
def plot_bollinger(df, figsize=(12, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='red')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='green')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.scatter(df.index[df['Buy_Signal_BB']], df['Close'][df['Buy_Signal_BB']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_BB']], df['Close'][df['Sell_Signal_BB']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Bollinger Bands')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    return fig

# Function to plot MACD with Buy/Sell Signals
def plot_macd(df, figsize=(12, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(df.index, df['MACD'], label='MACD', color='blue')
    ax.plot(df.index, df['MACD_Signal'], label='Signal Line', color='red')
    ax.bar(df.index, df['MACD'] - df['MACD_Signal'], color=df.apply(
        lambda x: 'green' if x['MACD'] > x['MACD_Signal'] else 'red', axis=1),
        alpha=0.5, label='Histogram')
    ax.set_title('Moving Average Convergence Divergence (MACD)')
    ax.set_ylabel('MACD Value')
    ax.legend()
    ax.grid(True, alpha=0.3)
    return fig

# Function to plot Moving Averages
def plot_moving_averages(df, figsize=(12, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(df.index, df['Close'], color='k', label='Close Price')
    ax.plot(df.index, df['50_SMA'], color='b', label='50-day SMA')
    ax.plot(df.index, df['200_SMA'], color='g', label='200-day SMA')

    # Highlight golden cross and death cross
    golden_cross = (df['50_SMA'] > df['200_SMA']) & (df['50_SMA'].shift() <= df['200_SMA'].shift())
    death_cross = (df['50_SMA'] < df['200_SMA']) & (df['50_SMA'].shift() >= df['200_SMA'].shift())

    if golden_cross.any():
        golden_dates = df.index[golden_cross]
        for date in golden_dates:
            ax.axvline(x=date, color='gold', linestyle='--', alpha=0.7)

    if death_cross.any():
        death_dates = df.index[death_cross]
        for date in death_dates:
            ax.axvline(x=date, color='darkred', linestyle='--', alpha=0.7)

    ax.set_title('Moving Averages (50-day and 200-day)')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    return fig

# Function to get technical analysis summary
def get_technical_summary(df):
    """Generate a summary dataframe of technical indicators"""
    latest = df.iloc[-1]

    signal_df = pd.DataFrame({
        'Indicator': ['RSI', 'Bollinger Bands', 'MACD', 'Moving Averages'],
        'Value': [
            f"{latest['RSI']:.2f}",
            f"Width: {((latest['BB_Upper'] - latest['BB_Lower'])/latest['Close']*100):.2f}%",
            f"MACD: {latest['MACD']:.4f}, Signal: {latest['MACD_Signal']:.4f}",
            f"50-day: {latest['50_SMA']:.2f}, 200-day: {latest['200_SMA']:.2f}"
        ],
        'Signal': [
            "Buy" if latest['Buy_Signal_RSI'] else "Sell" if latest['Sell_Signal_RSI'] else "Hold",
            "Buy" if latest['Buy_Signal_BB'] else "Sell" if latest['Sell_Signal_BB'] else "Hold",
            "Buy" if latest['Buy_Signal_MACD'] else "Sell",
            "Bullish" if latest['50_SMA'] > latest['200_SMA'] else "Bearish"
        ]
    })

    return signal_df

# Function to generate technical trading signals
def generate_combined_signal(df):
    """Generate a combined trading signal based on multiple indicators"""
    # Get the latest row
    latest = df.iloc[-1]

    # Count buy and sell signals
    buy_signals = 0
    sell_signals = 0

    # RSI
    if latest['Buy_Signal_RSI']:
        buy_signals += 1
    elif latest['Sell_Signal_RSI']:
        sell_signals += 1

    # Bollinger Bands
    if latest['Buy_Signal_BB']:
        buy_signals += 1
    elif latest['Sell_Signal_BB']:
        sell_signals += 1

    # MACD
    if latest['Buy_Signal_MACD']:
        buy_signals += 1
    else:
        sell_signals += 1

    # Moving Averages (Golden Cross / Death Cross)
    if latest['50_SMA'] > latest['200_SMA']:
        buy_signals += 1
    else:
        sell_signals += 1

    # Determine overall signal
    if buy_signals > sell_signals:
        return "BUY"
    elif sell_signals > buy_signals:
        return "SELL"
    else:
        return "HOLD"

# Plot market trend distribution
def plot_trend_distribution(df, figsize=(8, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    df['Trend'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax, colors=plt.cm.Paired.colors)
    plt.title('Distribution of Price Movements')
    return fig

Writing technical_indicators.py


In [4]:
%%writefile stockfinal.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import matplotlib.pyplot as plt

# Import our technical analysis module
from technical_indicators import (
    apply_all_indicators,
    plot_rsi,
    plot_bollinger,
    plot_macd,
    plot_moving_averages,
    get_technical_summary,
    generate_combined_signal,
    plot_trend_distribution
)

# Configure page
st.set_page_config(
    page_title="Stock Analysis & Forecast App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Analysis & Forecast App')
st.markdown("""
This app provides comprehensive stock analysis using technical indicators and forecasts future prices using Facebook's Prophet model.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()

        if period == '5y' or period == '1825d':
            start_date = end_date - timedelta(days=1825)
        else:
            # For technical analysis periods
            df = yf.download(ticker, period=period)
            return df

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            if period == '5y' or period == '1825d':
                return prepare_dataframe(df), df
            return df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        if period == '5y' or period == '1825d':
            return prepare_dataframe(sample_data), sample_data
        return sample_data

def display_technical_analysis(df, ticker):
    """Display technical analysis charts and data"""
    st.header(f"Technical Analysis for {ticker}")

    # Apply all technical indicators
    df_tech = apply_all_indicators(df)

    # Market Trend Analysis
    st.subheader("Market Trend Analysis")
    col1, col2 = st.columns(2)
    with col1:
        fig = plot_trend_distribution(df_tech)
        st.pyplot(fig)

    with col2:
        st.markdown("### Trend Interpretation")
        st.write("""
        This pie chart shows the distribution of daily price movements categorized by trend strength.
        - **Bull run/Bear drop**: Extreme price movements (>7%)
        - **Top gainers/losers**: Strong movements (3-7%)
        - **Positive/Negative**: Moderate movements (1-3%)
        - **Slight changes**: Minor movements (0.5-1%)
        - **No change**: Minimal price movement (<0.5%)
        """)

    # RSI with Buy/Sell Signals
    st.subheader("RSI Indicator with Buy/Sell Signals")
    fig_rsi = plot_rsi(df_tech)
    st.pyplot(fig_rsi)

    # Bollinger Bands with Buy/Sell Signals
    st.subheader("Bollinger Bands with Buy/Sell Signals")
    fig_bb = plot_bollinger(df_tech)
    st.pyplot(fig_bb)

    # MACD with Buy/Sell Signals
    st.subheader("MACD Indicator with Buy/Sell Signals")
    fig_macd = plot_macd(df_tech)
    st.pyplot(fig_macd)

    # Moving Averages
    st.subheader("Moving Averages")
    fig_ma = plot_moving_averages(df_tech)
    st.pyplot(fig_ma)

    # Technical Analysis Summary
    st.subheader("Technical Analysis Summary")

    latest = df_tech.iloc[-1]
    col1, col2, col3 = st.columns(3)

    with col1:
        rsi_status = "Overbought" if latest['RSI'] > 70 else "Oversold" if latest['RSI'] < 30 else "Neutral"
        st.metric("RSI Value", f"{latest['RSI']:.2f}", delta=rsi_status)

    with col2:
        bb_position = ((latest['Close'] - latest['BB_Lower']) / (latest['BB_Upper'] - latest['BB_Lower'])) * 100
        bb_status = "Upper Band" if bb_position > 80 else "Lower Band" if bb_position < 20 else "Middle"
        st.metric("Bollinger Position", f"{bb_position:.1f}%", delta=bb_status)

    with col3:
        macd_signal = "Bullish" if latest['MACD'] > latest['MACD_Signal'] else "Bearish"
        macd_diff = latest['MACD'] - latest['MACD_Signal']
        st.metric("MACD Signal", macd_signal, delta=f"{macd_diff:.4f}")

    # Technical signals summary
    signal_df = get_technical_summary(df_tech)
    st.table(signal_df)

    # Combined signal
    combined_signal = generate_combined_signal(df_tech)
    signal_color = "green" if combined_signal == "BUY" else "red" if combined_signal == "SELL" else "orange"

    st.markdown(f"""
    <div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px; margin-top: 20px;">
        <h3 style="text-align: center;">Overall Trading Signal</h3>
        <h1 style="text-align: center; color: {signal_color};">{combined_signal}</h1>
        <p style="text-align: center;">Based on combined analysis of multiple technical indicators</p>
    </div>
    """, unsafe_allow_html=True)

    # Data Preview
    st.subheader("Recent Data and Indicators")
    st.dataframe(df_tech.tail())

    return df_tech

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))
analysis_mode = st.sidebar.selectbox("Analysis Mode", ["Forecast", "Technical Analysis", "Both"])

# Technical analysis settings
if analysis_mode in ["Technical Analysis", "Both"]:
    st.sidebar.subheader("Technical Analysis Settings")
    ta_period = st.sidebar.selectbox(
        "Technical Analysis Period",
        ["1mo", "3mo", "6mo", "1y", "2y", "5y"],
        index=3
    )

# Forecast settings
if analysis_mode in ["Forecast", "Both"]:
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period = n_years * 365

# Main app flow
try:
    # Fetch data
    ticker = stocks[selected_stock]

    if analysis_mode == "Forecast":
        df_train, raw_data = fetch_stock_data(ticker)
        show_forecast = True
        show_technical = False
    elif analysis_mode == "Technical Analysis":
        df_tech = fetch_stock_data(ticker, ta_period)
        df_tech = display_technical_analysis(df_tech, selected_stock)
        show_forecast = False
        show_technical = True
    else:  # Both
        df_train, raw_data = fetch_stock_data(ticker)
        df_tech = fetch_stock_data(ticker, ta_period)
        df_tech = display_technical_analysis(df_tech, selected_stock)
        show_forecast = True
        show_technical = True

    # Display Forecast if selected
    if show_forecast:
        st.header("Stock Price Forecast")

        # Display metrics
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
        with col2:
            st.metric("Trading Days", len(df_train))
        with col3:
            price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
            st.metric("Total Return", f"{price_change:.1f}%")

        # Historical price chart
        st.subheader("Historical Price Data")
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_train["ds"],
            y=df_train["y"],
            name="Price",
            line=dict(color='royalblue', width=1.5)
        ))

        if len(df_train) >= 30:
            ma30 = df_train['y'].rolling(window=30).mean()
            fig.add_trace(go.Scatter(
                x=df_train["ds"][29:],
                y=ma30[29:],
                name="30-Day MA",
                line=dict(color='orange', width=1.5, dash='dot')
            ))

        fig.update_layout(
            title=f"{selected_stock} Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white",
            xaxis_rangeslider_visible=True
        )
        st.plotly_chart(fig, use_container_width=True)

        # Forecast
        st.subheader("Price Forecast")
        with st.spinner("Generating forecast..."):
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.05
            )
            m.fit(df_train)
            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)

        # Forecast results
        col1, col2 = st.columns(2)
        with col1:
            st.write("Latest Predictions")
            forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
            forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
            st.dataframe(forecast_tail)

        with col2:
            current_price = df_train['y'].iloc[-1]
            last_prediction = forecast['yhat'].iloc[-1]
            price_change = ((last_prediction - current_price) / current_price) * 100
            st.metric(
                "Predicted Change",
                f"{price_change:.1f}%",
                delta=f"${last_prediction - current_price:.2f}"
            )

        # Forecast plot
        fig1 = plot_plotly(m, forecast)
        fig1.update_layout(
            title=f"{selected_stock} Forecast - {n_years} Years",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        st.plotly_chart(fig1, use_container_width=True)

        # Components plot
        st.subheader("Forecast Components")
        fig2 = m.plot_components(forecast)
        st.write(fig2)

        # Disclaimer for forecast
        st.info("""
        📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
        not captured in this model. Always conduct thorough research and consult with financial advisors before making
        investment decisions.
        """)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit, Facebook Prophet, and Technical Indicators
""")

Writing stockfinal.py


In [7]:
!streamlit run  technical_indicators.py & npx localtunnel --port 8501

⠙⠹⠸

⠼⠴⠦⠧⠇⠏⠋⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.184.130:8501

⠹⠸your url is: https://clever-tips-ask.loca.lt
  Stopping...
^C


In [8]:
%%writefile stock.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go

# Configure page
st.set_page_config(
    page_title="Stock Forecast App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Forecast App')
st.markdown("""
This app forecasts future stock prices using Facebook's Prophet model.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()
        start_date = end_date - timedelta(days=1825)  # 5 years

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            return prepare_dataframe(df), df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        return prepare_dataframe(sample_data), sample_data

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))

# Forecast settings
st.sidebar.subheader("Forecast Settings")
n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
period = n_years * 365

# Main app flow
try:
    # Fetch data
    ticker = stocks[selected_stock]
    df_train, raw_data = fetch_stock_data(ticker)

    st.header("Stock Price Forecast")

    # Display metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
    with col2:
        st.metric("Trading Days", len(df_train))
    with col3:
        price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
        st.metric("Total Return", f"{price_change:.1f}%")

    # Historical price chart
    st.subheader("Historical Price Data")
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_train["ds"],
        y=df_train["y"],
        name="Price",
        line=dict(color='royalblue', width=1.5)
    ))

    if len(df_train) >= 30:
        ma30 = df_train['y'].rolling(window=30).mean()
        fig.add_trace(go.Scatter(
            x=df_train["ds"][29:],
            y=ma30[29:],
            name="30-Day MA",
            line=dict(color='orange', width=1.5, dash='dot')
        ))

    fig.update_layout(
        title=f"{selected_stock} Price History",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",
        xaxis_rangeslider_visible=True
    )
    st.plotly_chart(fig, use_container_width=True)

    # Forecast
    st.subheader("Price Forecast")
    with st.spinner("Generating forecast..."):
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05
        )
        m.fit(df_train)
        future = m.make_future_dataframe(periods=period)
        forecast = m.predict(future)

    # Forecast results
    col1, col2 = st.columns(2)
    with col1:
        st.write("Latest Predictions")
        forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
        forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
        st.dataframe(forecast_tail)

    with col2:
        current_price = df_train['y'].iloc[-1]
        last_prediction = forecast['yhat'].iloc[-1]
        price_change = ((last_prediction - current_price) / current_price) * 100
        st.metric(
            "Predicted Change",
            f"{price_change:.1f}%",
            delta=f"${last_prediction - current_price:.2f}"
        )

    # Forecast plot
    fig1 = plot_plotly(m, forecast)
    fig1.update_layout(
        title=f"{selected_stock} Forecast - {n_years} Years",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white"
    )
    st.plotly_chart(fig1, use_container_width=True)

    # Components plot
    st.subheader("Forecast Components")
    fig2 = m.plot_components(forecast)
    st.write(fig2)

    # Disclaimer for forecast
    st.info("""
    📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
    not captured in this model. Always conduct thorough research and consult with financial advisors before making
    investment decisions.
    """)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit and Facebook Prophet
""")

Writing stock.py


In [9]:
!streamlit run  stock.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.184.130:8501

⠦⠧⠇⠏⠋your url is: https://fine-rocks-kick.loca.lt
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
04:18:52 - cmdstanpy - INFO - Chain [1] start processing
04:18:52 - cmdstanpy - INFO - Chain [1] done processing
  Stopping...
^C


In [10]:
%%writefile technical_analysis.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Function to fetch stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

# Function to calculate trends
def trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

# Streamlit App
st.title("Stock Market Technical Analysis")
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, MSFT)", "AAPL")
period = st.selectbox("Select Time Period", ['1mo', '3mo', '6mo', '1y', '2y', '5y'], index=3)

if st.button("Analyze"):
    df = get_stock_data(ticker, period)
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))

    # Plot Trends
    st.subheader("Market Trend Analysis")
    fig, ax = plt.subplots()
    df['Trend'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax)
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    fig, ax = plt.subplots()
    df['Close'].plot(ax=ax, color='k', label='Close Price')
    df['50_SMA'].plot(ax=ax, color='b', label='50-day SMA')
    df['200_SMA'].plot(ax=ax, color='g', label='200-day SMA')
    ax.legend()
    st.pyplot(fig)

    # RSI Indicator
    st.subheader("Relative Strength Index (RSI)")
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    fig, ax = plt.subplots()
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red')
    ax.axhline(30, linestyle='--', color='green')
    ax.legend()
    st.pyplot(fig)

    # Bollinger Bands
    st.subheader("Bollinger Bands")
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Middle'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='k')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='r')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='g')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.legend()
    st.pyplot(fig)

    st.write("Data Preview:")
    st.dataframe(df.tail())


Writing technical_analysis.py


In [11]:
%%writefile stockfinal.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import importlib
import os

# Configure page
st.set_page_config(
    page_title="Stock Forecast App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Forecast App')
st.markdown("""
This app forecasts future stock prices using Facebook's Prophet model and provides technical analysis options.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()
        start_date = end_date - timedelta(days=1825)  # 5 years

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            return prepare_dataframe(df), df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        return prepare_dataframe(sample_data), sample_data

def load_technical_analysis_module():
    """Attempt to import technical_analysis.py if it exists"""
    if os.path.exists("technical_analysis.py"):
        try:
            import technical_analysis
            return technical_analysis
        except ImportError:
            st.warning("technical_analysis.py exists but could not be imported.")
    return None

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))

# Add analysis type selection
analysis_type = st.sidebar.selectbox(
    "Analysis Type",
    ["Forecast", "Technical Analysis", "Both"]
)

# Forecast settings
if analysis_type in ["Forecast", "Both"]:
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period = n_years * 365

# Technical analysis settings if needed
if analysis_type in ["Technical Analysis", "Both"]:
    st.sidebar.subheader("Technical Analysis Settings")
    ta_period = st.sidebar.selectbox(
        "Technical Analysis Period",
        ["1mo", "3mo", "6mo", "1y", "2y", "5y"],
        index=3
    )

# Main app flow
try:
    # Fetch data
    ticker = stocks[selected_stock]

    # Show technical analysis if selected
    if analysis_type in ["Technical Analysis", "Both"]:
        # Try to load technical analysis module
        ta_module = load_technical_analysis_module()

        if ta_module:
            st.header("Technical Analysis")
            # Get data for technical analysis
            if not 'df_tech' in locals():
                _, df_tech = fetch_stock_data(ticker)

            # Call functions from the technical analysis module
            ta_module.run_analysis(df_tech, ticker, ta_period)
        else:
            st.error("Technical Analysis module not found. Please make sure 'technical_analysis.py' is in the same directory.")

    # Show forecast if selected
    if analysis_type in ["Forecast", "Both"]:
        df_train, raw_data = fetch_stock_data(ticker)

        st.header("Stock Price Forecast")

        # Display metrics
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
        with col2:
            st.metric("Trading Days", len(df_train))
        with col3:
            price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
            st.metric("Total Return", f"{price_change:.1f}%")

        # Historical price chart
        st.subheader("Historical Price Data")
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_train["ds"],
            y=df_train["y"],
            name="Price",
            line=dict(color='royalblue', width=1.5)
        ))

        if len(df_train) >= 30:
            ma30 = df_train['y'].rolling(window=30).mean()
            fig.add_trace(go.Scatter(
                x=df_train["ds"][29:],
                y=ma30[29:],
                name="30-Day MA",
                line=dict(color='orange', width=1.5, dash='dot')
            ))

        fig.update_layout(
            title=f"{selected_stock} Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white",
            xaxis_rangeslider_visible=True
        )
        st.plotly_chart(fig, use_container_width=True)

        # Forecast
        st.subheader("Price Forecast")
        with st.spinner("Generating forecast..."):
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.05
            )
            m.fit(df_train)
            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)

        # Forecast results
        col1, col2 = st.columns(2)
        with col1:
            st.write("Latest Predictions")
            forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
            forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
            st.dataframe(forecast_tail)

        with col2:
            current_price = df_train['y'].iloc[-1]
            last_prediction = forecast['yhat'].iloc[-1]
            price_change = ((last_prediction - current_price) / current_price) * 100
            st.metric(
                "Predicted Change",
                f"{price_change:.1f}%",
                delta=f"${last_prediction - current_price:.2f}"
            )

        # Forecast plot
        fig1 = plot_plotly(m, forecast)
        fig1.update_layout(
            title=f"{selected_stock} Forecast - {n_years} Years",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        st.plotly_chart(fig1, use_container_width=True)

        # Components plot
        st.subheader("Forecast Components")
        fig2 = m.plot_components(forecast)
        st.write(fig2)

        # Disclaimer for forecast
        st.info("""
        📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
        not captured in this model. Always conduct thorough research and consult with financial advisors before making
        investment decisions.
        """)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit and Facebook Prophet
""")

Overwriting stockfinal.py


In [15]:
!streamlit run  stockfinal.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.184.130:8501

⠴⠦your url is: https://clean-waves-accept.loca.lt
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
04:29:32 - cmdstanpy - INFO - Chain [1] start processing
04:29:33 - cmdstanpy - INFO - Chain [1] done processing
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
04:30:24 - cmdstanpy - INFO - Chain [1] start processing
04:30:24 - cmdstanpy - INFO - Chain [1] done processing
  Stopping...
^C


In [14]:
%%writefile stockfinal.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Configure page
st.set_page_config(
    page_title="Stock Analysis App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Analysis App')
st.markdown("""
This app provides stock forecasting using Facebook's Prophet model and technical analysis tools.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()
        start_date = end_date - timedelta(days=1825)  # 5 years

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            return prepare_dataframe(df), df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        return prepare_dataframe(sample_data), sample_data

# Technical Analysis Functions
def trend(x):
    """Calculate trend based on percentage change"""
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

def technical_analysis(df, ticker, period):
    """Perform technical analysis on the stock data"""
    # Calculate percentage change
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))

    # Plot Trends
    st.subheader("Market Trend Analysis")
    fig, ax = plt.subplots(figsize=(8, 6))
    trend_counts = df['Trend'].value_counts()
    trend_counts.plot.pie(autopct='%1.1f%%', ax=ax)
    ax.set_title(f"{ticker} - Trend Distribution")
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['Close'], color='k', label='Close Price')
    ax.plot(df.index, df['50_SMA'], color='b', label='50-day SMA')
    ax.plot(df.index, df['200_SMA'], color='g', label='200-day SMA')
    ax.set_title(f"{ticker} - Moving Averages")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    st.pyplot(fig)

    # RSI Indicator
    st.subheader("Relative Strength Index (RSI)")
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    ax.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    ax.set_title(f"{ticker} - RSI")
    ax.set_xlabel("Date")
    ax.set_ylabel("RSI Value")
    ax.legend()
    st.pyplot(fig)

    # Bollinger Bands
    st.subheader("Bollinger Bands")
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Middle'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['Close'], label='Close Price', color='k')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='r')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='g')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1, color='blue')
    ax.set_title(f"{ticker} - Bollinger Bands")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    st.pyplot(fig)

    # Data Preview
    st.subheader("Data Preview")
    st.dataframe(df.tail())

def forecast_analysis(df_train, raw_data, ticker, selected_stock, n_years):
    """Perform forecast analysis on the stock data"""
    period = n_years * 365

    st.header("Stock Price Forecast")

    # Display metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
    with col2:
        st.metric("Trading Days", len(df_train))
    with col3:
        price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
        st.metric("Total Return", f"{price_change:.1f}%")

    # Historical price chart
    st.subheader("Historical Price Data")
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_train["ds"],
        y=df_train["y"],
        name="Price",
        line=dict(color='royalblue', width=1.5)
    ))

    if len(df_train) >= 30:
        ma30 = df_train['y'].rolling(window=30).mean()
        fig.add_trace(go.Scatter(
            x=df_train["ds"][29:],
            y=ma30[29:],
            name="30-Day MA",
            line=dict(color='orange', width=1.5, dash='dot')
        ))

    fig.update_layout(
        title=f"{selected_stock} Price History",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",
        xaxis_rangeslider_visible=True
    )
    st.plotly_chart(fig, use_container_width=True)

    # Forecast
    st.subheader("Price Forecast")
    with st.spinner("Generating forecast..."):
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05
        )
        m.fit(df_train)
        future = m.make_future_dataframe(periods=period)
        forecast = m.predict(future)

    # Forecast results
    col1, col2 = st.columns(2)
    with col1:
        st.write("Latest Predictions")
        forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
        forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
        st.dataframe(forecast_tail)

    with col2:
        current_price = df_train['y'].iloc[-1]
        last_prediction = forecast['yhat'].iloc[-1]
        price_change = ((last_prediction - current_price) / current_price) * 100
        st.metric(
            "Predicted Change",
            f"{price_change:.1f}%",
            delta=f"${last_prediction - current_price:.2f}"
        )

    # Forecast plot
    fig1 = plot_plotly(m, forecast)
    fig1.update_layout(
        title=f"{selected_stock} Forecast - {n_years} Years",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white"
    )
    st.plotly_chart(fig1, use_container_width=True)

    # Components plot
    st.subheader("Forecast Components")
    fig2 = m.plot_components(forecast)
    st.write(fig2)

    # Disclaimer for forecast
    st.info("""
    📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
    not captured in this model. Always conduct thorough research and consult with financial advisors before making
    investment decisions.
    """)

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))
ticker = stocks[selected_stock]

# Analysis type selection
analysis_type = st.sidebar.selectbox("Select Analysis Type", ["Price Forecast", "Technical Analysis"])

# Period selection (different options for different analysis types)
if analysis_type == "Price Forecast":
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period_forecast = "5y"  # Always use 5y for forecast
else:  # Technical Analysis
    st.sidebar.subheader("Technical Analysis Settings")
    period_options = {'1 Month': '1mo', '3 Months': '3mo', '6 Months': '6mo', '1 Year': '1y', '2 Years': '2y', '5 Years': '5y'}
    selected_period = st.sidebar.selectbox("Select Time Period", list(period_options.keys()), index=3)
    period_tech = period_options[selected_period]

# Main app flow
try:
    if analysis_type == "Price Forecast":
        # Fetch data for forecasting
        df_train, raw_data = fetch_stock_data(ticker, period_forecast)
        forecast_analysis(df_train, raw_data, ticker, selected_stock, n_years)
    else:  # Technical Analysis
        # Fetch data for technical analysis
        with st.spinner(f"Fetching data for {ticker}..."):
            df = fetch_data_from_yfinance(ticker, period_tech)
            if df is None or df.empty or len(df) <= 30:
                st.warning(f"Using sample data for {ticker}")
                df = get_sample_data(ticker)
            technical_analysis(df, selected_stock, period_tech)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit, Facebook Prophet, and TA-Lib
""")

Overwriting stockfinal.py


In [19]:
%%writefile stockfinal2.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Configure page
st.set_page_config(
    page_title="Stock Analysis App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Analysis App')
st.markdown("""
This app provides stock forecasting using Facebook's Prophet model and technical analysis tools.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()
        start_date = end_date - timedelta(days=1825)  # 5 years

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            return prepare_dataframe(df), df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        return prepare_dataframe(sample_data), sample_data

# Technical Analysis Functions
def trend(x):
    """Calculate trend based on percentage change"""
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

def technical_analysis(df, ticker, period):
    """Perform technical analysis on the stock data"""
    # Ensure we're working with a dataframe that has Close as a column, not a series
    if isinstance(df, pd.Series):
        df = df.to_frame(name='Close')

    # Calculate percentage change
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))

    # Plot Trends
    st.subheader("Market Trend Analysis")
    fig, ax = plt.subplots(figsize=(8, 6))
    trend_counts = df['Trend'].value_counts()
    trend_counts.plot.pie(autopct='%1.1f%%', ax=ax)
    ax.set_title(f"{ticker} - Trend Distribution")
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['Close'], color='k', label='Close Price')
    ax.plot(df.index, df['50_SMA'], color='b', label='50-day SMA')
    ax.plot(df.index, df['200_SMA'], color='g', label='200-day SMA')
    ax.set_title(f"{ticker} - Moving Averages")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    st.pyplot(fig)

    # RSI Indicator
    st.subheader("Relative Strength Index (RSI)")
    # Create a clean DataFrame for the RSI calculation
    df_rsi = pd.DataFrame(index=df.index)
    df_rsi['close'] = df['Close']
    # Use the ta-lib RSI function with the clean DataFrame
    rsi_indicator = RSIIndicator(close=df_rsi['close'], window=14)
    df['RSI'] = rsi_indicator.rsi()

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    ax.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    ax.set_title(f"{ticker} - RSI")
    ax.set_xlabel("Date")
    ax.set_ylabel("RSI Value")
    ax.legend()
    st.pyplot(fig)

    # Bollinger Bands
    st.subheader("Bollinger Bands")
    # Create a clean DataFrame for the Bollinger Bands calculation
    df_bb = pd.DataFrame(index=df.index)
    df_bb['close'] = df['Close']
    # Use the ta-lib Bollinger Bands function with the clean DataFrame
    bollinger = BollingerBands(close=df_bb['close'], window=20, window_dev=2)
    df['BB_Middle'] = bollinger.bollinger_mavg()
    df['BB_Upper'] = bollinger.bollinger_hband()
    df['BB_Lower'] = bollinger.bollinger_lband()

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df.index, df['Close'], label='Close Price', color='k')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='r')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='g')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1, color='blue')
    ax.set_title(f"{ticker} - Bollinger Bands")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    st.pyplot(fig)

    # Data Preview
    st.subheader("Data Preview")
    st.dataframe(df.tail())

def forecast_analysis(df_train, raw_data, ticker, selected_stock, n_years):
    """Perform forecast analysis on the stock data"""
    period = n_years * 365

    st.header("Stock Price Forecast")

    # Display metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
    with col2:
        st.metric("Trading Days", len(df_train))
    with col3:
        price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
        st.metric("Total Return", f"{price_change:.1f}%")

    # Historical price chart
    st.subheader("Historical Price Data")
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_train["ds"],
        y=df_train["y"],
        name="Price",
        line=dict(color='royalblue', width=1.5)
    ))

    if len(df_train) >= 30:
        ma30 = df_train['y'].rolling(window=30).mean()
        fig.add_trace(go.Scatter(
            x=df_train["ds"][29:],
            y=ma30[29:],
            name="30-Day MA",
            line=dict(color='orange', width=1.5, dash='dot')
        ))

    fig.update_layout(
        title=f"{selected_stock} Price History",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",
        xaxis_rangeslider_visible=True
    )
    st.plotly_chart(fig, use_container_width=True)

    # Forecast
    st.subheader("Price Forecast")
    with st.spinner("Generating forecast..."):
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05
        )
        m.fit(df_train)
        future = m.make_future_dataframe(periods=period)
        forecast = m.predict(future)

    # Forecast results
    col1, col2 = st.columns(2)
    with col1:
        st.write("Latest Predictions")
        forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
        forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
        st.dataframe(forecast_tail)

    with col2:
        current_price = df_train['y'].iloc[-1]
        last_prediction = forecast['yhat'].iloc[-1]
        price_change = ((last_prediction - current_price) / current_price) * 100
        st.metric(
            "Predicted Change",
            f"{price_change:.1f}%",
            delta=f"${last_prediction - current_price:.2f}"
        )

    # Forecast plot
    fig1 = plot_plotly(m, forecast)
    fig1.update_layout(
        title=f"{selected_stock} Forecast - {n_years} Years",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white"
    )
    st.plotly_chart(fig1, use_container_width=True)

    # Components plot
    st.subheader("Forecast Components")
    fig2 = m.plot_components(forecast)
    st.write(fig2)

    # Disclaimer for forecast
    st.info("""
    📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
    not captured in this model. Always conduct thorough research and consult with financial advisors before making
    investment decisions.
    """)

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))
ticker = stocks[selected_stock]

# Analysis type selection
analysis_type = st.sidebar.selectbox("Select Analysis Type", ["Price Forecast", "Technical Analysis"])

# Period selection (different options for different analysis types)
if analysis_type == "Price Forecast":
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period_forecast = "5y"  # Always use 5y for forecast
else:  # Technical Analysis
    st.sidebar.subheader("Technical Analysis Settings")
    period_options = {'1 Month': '1mo', '3 Months': '3mo', '6 Months': '6mo', '1 Year': '1y', '2 Years': '2y', '5 Years': '5y'}
    selected_period = st.sidebar.selectbox("Select Time Period", list(period_options.keys()), index=3)
    period_tech = period_options[selected_period]

# Main app flow
try:
    if analysis_type == "Price Forecast":
        # Fetch data for forecasting
        df_train, raw_data = fetch_stock_data(ticker, period_forecast)
        forecast_analysis(df_train, raw_data, ticker, selected_stock, n_years)
    else:  # Technical Analysis
        # Fetch data for technical analysis
        with st.spinner(f"Fetching data for {ticker}..."):
            df = fetch_data_from_yfinance(ticker, period_tech)
            if df is None or df.empty or len(df) <= 30:
                st.warning(f"Using sample data for {ticker}")
                df = get_sample_data(ticker)
            technical_analysis(df, selected_stock, period_tech)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    with st.expander("Error details"):
        st.code(str(e))
        import traceback
        st.code(traceback.format_exc())

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit, Facebook Prophet, and TA-Lib
""")

Overwriting stockfinal2.py


In [20]:
!streamlit run  stockfinal2.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.184.130:8501

⠦your url is: https://smooth-hoops-relax.loca.lt
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
04:39:10 - cmdstanpy - INFO - Chain [1] start processing
04:39:11 - cmdstanpy - INFO - Chain [1] done processing
[*********************100%***********************]  1 of 1 completed
  Stopping...
^C


Above stockfinal2 project is working well with technical analysis